In [17]:
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
from requests import get

import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

#url = 'https://www.imdb.com/list/ls095374765/?sort=list_order,asc&st_dt=&mode=grid&page=1&ref_=ttls_ref_gnr&genres=Drama'
url = 'https://www.imdb.com/list/ls095374765/?sort=moviemeter,asc&st_dt=&mode=grid&page=1&ref_=ttls_vw_smp&genres=Comedy'
response = get(url)
html_soupMain = BeautifulSoup(response.text, 'html.parser')

In [18]:
movieReview = []
reviewTitle = []
movieTitle = []

In [19]:
def list_Of_Reviews(website):
    soup = BeautifulSoup(website, 'html.parser')
    # print(soup)
    f = soup.find_all('div', class_='ipl-icon ipl-star-icon')
    f = soup.find_all('span', class_="rating-other-user-rating")#.find('span').text

    idxPos = -1
    idxNeg = -1

    for idx, i in enumerate(f):
        sp = (f[idx].text.split('/'))
        sp = int(sp[0][-2:])
        if sp > 8:
            idxPos = idx
        elif sp < 4:
            idxNeg = idx
    return([idxNeg,idxPos])

In [20]:
stop_words = set(stopwords.words("english"))
for div in html_soupMain.findAll('div', attrs={'class':'title'}):
    if len(movieReview) > 98:
        break
    else:
        url = 'https://www.imdb.com' + div.find('a')['href'] 
        url = url + 'reviews'
        response = get(url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        review_containers = html_soup.find_all('div', class_='text show-more__control')
        title_containers = html_soup.find_all( class_='title')
        
        reviewIndex = list_Of_Reviews(response.content)
        reviewList = []
        if reviewIndex[0] == -1 or reviewIndex[1] == -1:
            pass
        else:
            reviewList.append(review_containers[reviewIndex[0]].text)
            reviewList.append(review_containers[reviewIndex[1]].text)
            reviewTitle.append(title_containers[reviewIndex[0]].string + ' -')
            reviewTitle.append(title_containers[reviewIndex[1]].string + ' +')
            movieTitle.append(div.string + ' -')
            movieTitle.append(div.string + ' +')
            count =+ 2
            for i in reviewList:
                review = i.lower()
                #remove punctuation
                review = re.sub('[^a-zA-Z]', ' ', review)
                
                #remove tags
                review = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",review)

                #remove special characters and digits
                review = re.sub("(\\d|\\W)+"," ",review)

                split_text = review.split()
                
                #Lemmatisation
                lem = WordNetLemmatizer()
                split_text = [lem.lemmatize(word) for word in split_text if not word in stop_words and len(word) >2] 
                split_text = " ".join(split_text)
                movieReview.append(split_text)

In [21]:
print(len(movieReview))

100


In [22]:
#TF-IDF vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(stop_words = {'english'})#stop_words, ngram_range = (1,1))#transform
vec_text = tfv.fit_transform(movieReview)#returns a list of words.
words = tfv.get_feature_names()

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

Sum_of_squared_distances = []
X = vec_text
K = range(2,10)
for k in K:
    km = KMeans(n_clusters=k, max_iter=200, n_init=10)
    km = km.fit(X)
    Sum_of_squared_distances.append(km.inertia_)
    
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

## Running with K = 8 centriods

In [24]:
import pandas as pd
true_k = 8
model = KMeans(n_clusters=true_k, max_iter=200, n_init=10, tol = 0.001)
model.fit(X)
labels=model.labels_
reviews_cl=pd.DataFrame(list(zip(movieTitle, reviewTitle,labels)),columns=['Movie','Review title','cluster'])
reviews_cl.sort_values(by=['cluster'])

,Movie,Review title,cluster
66,Captain Ron -,Purely enjoyable entertainment\n -,0
76,Canadian Bacon -,NOT funny. Better watch a regular Michael Moo...,0
2,Sky High -,"Above Average, Rough Around the Edges Family ...",0
43,My Super Ex-Girlfriend +,Not exactly super\n +,0
68,Funny Farm -,Underrated Chase sleeper\n -,0
...,...,...,...
46,Inspector Gadget -,"Not as bad as some say, but it could have bee...",7
83,Blind Fury +,One great movie!\n +,7
35,Spies Like Us +,"Past its sell-by, but still pretty good.\n +",7
92,Armed and Dangerous -,Candy Time!\n -,7


In [25]:
common_words = model.cluster_centers_.argsort()[:,-1:-11:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))

0 : zorro, movie, good, russell, nice, ron, captain, entertaining, short, character
1 : vacation, christmas, loved, vega, audrey, light, bit, especially, clark, europe
2 : candy, aykroyd, chet, family, john, daryl, aussie, movie, roman, vacation
3 : film, one, comedy, really, hank, character, movie, funny, quite, fun
4 : movie, great, make, kid, watching, joke, titan, funny, one, like
5 : ninja, haru, hill, beverly, farley, chris, film, shou, boyfriend, adopted
6 : first, movie, one, scene, time, part, deathstalker, like, watch, way
7 : john, candy, levy, good, comedy, film, classic, movie, eugene, rental


## Running with K = 6 centriods

In [26]:
import pandas as pd
true_k = 8
model = KMeans(n_clusters=true_k, max_iter=200, n_init=10, tol = 0.001)
model.fit(X)
labels=model.labels_
reviews_cl=pd.DataFrame(list(zip(movieTitle, reviewTitle,labels)),columns=['Movie','Review title','cluster'])
reviews_cl.sort_values(by=['cluster'])

,Movie,Review title,cluster
1,Hercules +,Jerk-ules\n +,0
17,The 'Burbs +,"Likable enough, but not great\n +",0
46,Inspector Gadget -,"Not as bad as some say, but it could have bee...",0
74,Delirious -,Not funny\n -,1
36,National Lampoon's European Vacation -,Some funny stuff here!\n -,1
...,...,...,...
86,Beverly Hills Ninja -,"So stupid, but amusing at times\n -",6
66,Captain Ron -,Purely enjoyable entertainment\n -,7
84,Justice League vs. Teen Titans -,Unbelievably uninspired\n -,7
67,Captain Ron +,Not on my boat you don't\n +,7


In [27]:
common_words = model.cluster_centers_.argsort()[:,-1:-11:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))

0 : effort, disney, produced, year, rank, tom, hank, nominated, sadly, animated
1 : dolph, action, wild, lundgren, manly, funny, watching, interest, transvestite, regret
2 : candy, vacation, john, aykroyd, chet, rental, summer, levy, doctor, roman
3 : film, comedy, movie, one, first, good, time, fun, scene, quite
4 : movie, film, kid, character, like, made, great, one, even, make
5 : batman, funny, movie, marvel, element, bad, series, feature, animated, pesci
6 : ninja, haru, beverly, hill, farley, chris, duck, mind, jones, howard
7 : ron, loved, russell, christmas, light, captain, short, movie, especially, bit
